This tutorial can partly be executed with the kernel 'kmc_workshop' directly in the code-cells of this jupyter-notebook.

**For parts of this tutorial, we ask you to use the terminal. Open the terminal via the dashboard (return to it by right-clicking "jupyterhub" in the top left corner, then select "open link in new window") -> new -> Other: Terminal**


# Objective

The aim of this tutorial is to give you an outlook, how you can work with metagenomic data.

This time, we will not work with the earlier processed 16S dataset. Instead we will create a new dataset with that originates from metagenomic shot-gun reads.

In addition to the method of amplicon sequencing, in which in our case areas in the ribosomal DNA of the 16S subunit are sequenced, there is also another method: Shotgun sequencing. With shotgun sequencing all DNA of the samples is sequenced and thus also full genomes for the microorganisms are present. This method, now called metagenomics, allows significantly more analysis options than what is possible with 16S data. This not only means a higher price per sample for sequencing, but also requires more computing power to process and ultimately analyze significantly larger data amounts. Since everything has been sequenced, the data naturally also contains reads that code for the 16S rRNA subunit. Therefore, in principle, it is also possible to use these as marker sequences to estimate taxa in the dataset, analogous to the pure 16S microbiome dataset. But these reads will only make up a tiny fraction (>1%) of the dataset, so we cannot recommend such use. Instead, there are many more clade-specific marker genes, which means that many more reads can be used for taxonomic determination in metagenomics.



# Practical part

You will notice in the next steps that it is a little more complicated to work with metagenomes. We work with many different programs instead of R. This means that you have to read up on exactly what each program can do and install them all. Below we will give you a brief overview of how the first steps working with metagenomes look like.

## Quality control

If you want to work with metagenome data, you have to follow a few steps, similar to 16S data. Here, too, a quality control must first be carried out.


To get to know the necessary QC steps, we will download a publicly available metagenome. We have already prepared this, but you could also download it with the following command if it is not available.

It is necessary that we leave the Jupyter notebook and get to know the terminal of a Linux computer. JupyterHub also offers you this. Open a new tab while clicking on the jupterhub logo in the top left. In the JupyterHub, the terminal can be opened in the dashboard (where you can see all files structured,), click on the button at the top right labeled “New” and then select “Terminal” in the dropdown menu at the bottom. 

In the terminal activate the conda environment, in which we installed all the required tools for the next steps:

In [ ]:
conda activate metagenome

If this has worked, the now active conda environment should be named before your username@biomedinf in the terminal window. Conda is a tool that allows you to create a so-called virtual environment so that tools and their dependencies do not interfere with other programs on the machine. That way we can generate reproducible results.

Now we will create a new folder and go into it so that we can start the actual metagenome processing:

In [ ]:
mkdir ~/kmc_workshop/metagenome && cd ~/kmc_workshop/metagenome

A real metagenome data can be quite huge and therefor need lots of time and computational capacities to be processed.

So we fill this directory with our dummy metagenome. Now we can actually work with it:

In [ ]:
cd ~/kmc_workshop/metagenome #make sure you are in the right directory

ln -s ~/kmc_workshop/inputs/metagenome/SRR5935872_2.fastq.gz ~/kmc_workshop/metagenome/MSM79H87_R2.fastq.gz && ln -s ~/kmc_workshop/inputs/metagenome/SRR5935872_1.fastq.gz ~/kmc_workshop/metagenome/MSM79H87_R1.fastq.gz

Now we assess the quality once before we start with the QC, we do this with the tool fastqc, which can generate an html output for each fastq file. 

In [ ]:
cd ~/kmc_workshop/metagenome

fastqc --quiet --threads 2 MSM79H87_R1.fastq.gz MSM79H87_R2.fastq.gz

You will find the html-output for each fastq file in the directory as we are: ~/kmc_workshop/metagenome

First of all, the individual reads are examined for their sequencing quality. Particularly at the edges, the quality may not be good enough in some cases. For this we can use programs that automatically trim away these poorly sequenced bases. Additionally, the tool also checks which reads are actually paired and if there are some reads that are unpaired. Unpaired reads are separated into their own file.

In [ ]:
cd ~/kmc_workshop/metagenome

bbduk.sh stats=MSM79H87.bbduk_adapter_stats threads=2 in=MSM79H87_R1.fastq.gz in2=MSM79H87_R2.fastq.gz out1=MSM79H87_R1_trimmed.fastq.gz out2=MSM79H87_R2_trimmed.fastq.gz outs=MSM79H87_unpaired_trimmed.fastq.gz ref=~/kmc_workshop/inputs/metagenome/bbmap/nextera.fa.gz ktrim=r k=23 mink=11 hdist=1 minlength=50 tpe tbo

Our new outputs now have a 'trimmed' in the name, these are now further qced by removing not wanted reads in our files. These could be artifacts and so-called PhiX reads, that can be present during sequencing. Phix reads are internal controls during sequencing that should actually be removed during data export, but can still be found in some data. These should be removed in any case. We do this step twice, once for the paired reads, and one time for the sequences that are not paired.

In [ ]:
cd ~/kmc_workshop/metagenome

bbduk.sh stats=MSM79H87.bbduk.artifacts.stats threads=2 in=MSM79H87_R1_trimmed.fastq.gz in2=MSM79H87_R2_trimmed.fastq.gz k=31 ref=artifacts,phix ordered cardinality out1=MSM79H87_R1_cleanwithhost.fastq.gz out2=MSM79H87_R2_cleanwithhost.fastq.gz minlength=50

bbduk.sh threads=2 in=MSM79H87_unpaired_trimmed.fastq.gz  k=31 ref=artifacts,phix ordered cardinality out1=MSM79H87_unpaired_cleanwithhost.fastq.gz minlength=50

Of course, stool samples also contain cells from the host, and therefore also their DNA. We try to remove this before sequencing so that we don't sequence something that we have to remove directly from our data. This is an important step as metagenome data with human reads inside can be traced back to their origin. This could cause some (legal) trouble if you want to publish your metagenomes at some point. 

To remove those, we use an aligner tool called Bowtie2. We align a human reference genome to our data and keep only reads that don't align. Everything else will be discarded.

In [ ]:
cd ~/kmc_workshop/metagenome

bowtie2 -x ~/kmc_workshop/inputs/metagenome/genome -1 MSM79H87_R1_cleanwithhost.fastq.gz -2 MSM79H87_R2_cleanwithhost.fastq.gz -U MSM79H87_unpaired_cleanwithhost.fastq.gz -S /dev/null --no-unal -p 2 --un-gz MSM79H87_unpaired_clean.fastq.gz --un-conc-gz MSM79H87_R%_clean.fastq.gz 2> MSM79H87_bowtie2_log.txt

We have now completed the most important steps of a qc. We can now use fastqc to inspect our finished reads more closely. This works as done before, just with different input files.

In [ ]:
cd ~/kmc_workshop/metagenome

fastqc --quiet--threads 2 MSM79H87_R1_clean.fastq.gz MSM79H87_R2_clean.fastq.gz

**TASK** Do you notice a difference between the fastqc reports from prior and after QC?

If everything worked, only a few reads should have been removed. And there should be hardly any other differences. This is simply because public metagenomes have often already been QCed, as in this case. But that doesn't matter, the data can be QCed twice, just to be sure you are working with QCed data in the end.

## Abundance estimation

As already mentioned, metagenomes are also useful for determining taxonomic abundances. There are a large number of programs with which this can be estimated. For example, there are MetaPhlAn, Kraken2, Salmon and many more that can do this. For this, you will always need a reference database that contains the markers. Therefore, you are always limited to the fact that you cannot find a thing if it is not stored in the database.

We will use a rather new released tool for this, it's called Sylph. Sylph is an extremely fast and memory efficient program for profiling and searching metagenomic samples against databases. You can find the preprint for how it actually works [here](https://www.biorxiv.org/content/10.1101/2023.11.20.567879v1.full). For us important is just, that it needs two steps that we need to perform and that we have prepared a database to query against. We use in this case a well-maintained database, that is called GENOME TAXONOMY DATABASE (GTDB). Sylph transforms the database of reference genomes and the metagenomes into subsampled k-mers which are then compares against.

In [ ]:
cd ~/kmc_workshop/metagenome

sylph sketch -1 MSM79H87_R1_clean.fastq.gz -2 MSM79H87_R2_clean.fastq.gz -t 2

In the first step the metagenomes are so-called sketched, this results in "bags of k-mers". From this the tool then can later quickly estimate the average nucleotide identity (ANI) against the genomes in the database, which happens in the second step:

In [ ]:
cd ~/kmc_workshop/metagenome

sylph profile -u --read-seq-id 99.1 ~/kmc_workshop/inputs/metagenome/gtdb-r220-c200-dbv1.syldb MSM79H87_R1_clean.fastq.gz.paired.sylsp -o  MSM79H87_sylph_profile.tbl -t 2

The output is called 'MSM79H87_sylph_profile.tbl' and you can inspect it. You read the sylph profile with e.g. the command-line tool less:

In [ ]:
less /dpool/bioinformatics_rawdata/kmc_workshop/metagenome/MSM79H87_sylph_profile.tbl

You can scroll with the arrow keys and exit the file with the 'q' key on your keyboard.

## From Abundance table to a Phyloseq object
Finally, you can now build a phyloseq object from such an abundance profile, with which you can use all the methods you have learned before.

First we load all the libraries that we used earlier:

In [ ]:
library(tidyverse) 
library(phyloseq)
library(vegan) 
library(microbiome) 
library(data.table)

In this case we use the abundance table of the single full metagenome sample from above:

In [ ]:
sylph <- read_delim("~/kmc_workshop/inputs/metagenome/MSM79H87_sylph_profile.tbl")
head(sylph)

We now create an abundance matrix from the sylph abundance table:

In [ ]:
# The GTDB taxonomic table is part of the GTDB release, that you can download here: https://ecogenomics.github.io/GTDBTk/installing/index.html
gtdb_taxa <- read_delim(
  "~/kmc_workshop/inputs/metagenome/bac120_taxonomy_r220.tsv.gz",
   col_names = c("genome_id","taxa_name"),delim = "\t") %>%
                mutate(genome_id=gsub("^[A-Z]+_","",genome_id))
head(gtdb_taxa)

sylph_withtaxa <- sylph %>% 
  select(genome_id=Genome_file,Sample=Sample_file, Abundance=Taxonomic_abundance) %>% 
  mutate(Sample=gsub("_R_R1_clean.fastq.gz","",Sample), 
         genome_id=gsub("_genomic.fna.gz","",basename(genome_id))) %>%
  left_join(gtdb_taxa, by="genome_id")

head(sylph_withtaxa)

In [ ]:
abundance_df <- sylph_withtaxa %>% 
  select(Taxa = taxa_name, Abundance, Sample) %>% 
  group_by(Sample, Taxa) %>%
  summarise(Abundance = sum(Abundance))


abundance_matrix <- abundance_df %>%
  pivot_wider(
    names_from = Sample, 
    values_from = c(Abundance),
    values_fill = 0) %>%
  column_to_rownames(var = "Taxa") %>%
  as.matrix()
head(abundance_matrix)

From the taxa_names column in our sylph_withtaxa object we can now create a taxonomic rank matrix by splitting the strings in this column by the delimiter and putting the splitted parts into an empty matrix.

In [ ]:
taxsplit <- strsplit(rownames(abundance_matrix), split=";", fixed = TRUE)
taxunitmatrix <- matrix(
    NA, 
    ncol = max(sapply(taxsplit, length)), 
    nrow = length(taxsplit))

colnames(taxunitmatrix) <- c(
    "Kingdom",
    "Phylum",
    "Class",
    "Order", 
    "Family",
    "Genus",
    "Species", 
    "Strain")[1:ncol(taxunitmatrix)]
rownames(taxunitmatrix) <- rownames(abundance_matrix)

for (i in 1:nrow(taxunitmatrix)){
    taxunitmatrix[i, 1:length(taxsplit[[i]])] <- taxsplit[[i]]
}
taxunitmatrix <- gsub("[a-z]__", "", taxunitmatrix)

rownames(taxunitmatrix) <- taxunitmatrix[,"Species"]

taxunitmatrix %>% head()

rownames(abundance_matrix) <- taxunitmatrix[,"Species"]

head(abundance_matrix)

We also need a metadata table for our samples. This could be created as follows:

In [ ]:
samples <- data.frame(SampleID="MSM79H87",metadata='some_meta_data') %>% 
  mutate(ID=SampleID) %>% 
  column_to_rownames("SampleID")
head(samples)

Now we have everything together to create a phyloseq object with which we can analyze further.

In [ ]:
otu_table_ps <- otu_table(abundance_matrix, taxa_are_rows = TRUE)
tax_table_ps <- tax_table(taxunitmatrix)
sample_data_ps <- sample_data(samples)
physeq <- phyloseq(tax_table_ps, otu_table_ps, sample_data_ps)

physeq

We take one last look at a stacked bar plot and see that we can now in principle apply everything, we learned earlier onto this phyloseq object containing metagenomic data!

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12, repr.plot.res = 100) #Set the plot size

physeq %>% 
  tax_glom("Phylum") %>%
  plot_bar(., x="Sample", fill="Phylum")+
  theme(legend.position="right")+ 
  guides(fill=guide_legend(nrow=5, byrow=TRUE))

**With that we want to close our workshop!**

If you have made it this far, we thank you very much for your attention and hope that you have learned a lot about how to work with microbiome data. If you have any questions after the workshop, feel free to write us an email, the addresses can be found above under the title of the respective notebook

In [ ]:
sessionInfo()